In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

In [39]:
pd.set_option('display.max_columns', None)
data = pd.read_csv('../data/data_teams.csv')
data.head()

,Unnamed: 0,Date,Time,TeamH,TeamA,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HStr,HSOH,HSOA,HSAH,HSAA,HSDH,HSDA,HO,HA,HM,HD,HDP,HIP,HSAAg,HATAAg,HGls,HAssts,HGlsPK,HxG,HNpxG,HxAG,HPoss,HPrg,HPrgP,HGKS%,HGKCS%,HSoT,Hg-xG,HCmp,HCmp%,HPrgDist,HXa,HKP,HPPA,HCrsPA,HTB,HCrs,HCK,HSCA,HGCA,HTkIW,HLost,HBlocks,HPass,HInt,HClr,HAttPen,HSucc%,HCPA,HDis,HPrgR,HOnG,HOnGA,HOnxG,HOnxGA,AStr,ASOH,ASOA,ASAH,ASAA,ASDH,ASDA,AO,AA,AM,AD,ADP,AIP,ASAAg,ATAAg,AGls,AAssts,AGlsPK,AxG,ANpxG,AxAG,APoss,APrg,APrgP,AGKS%,AGKCS%,ASoT,Ag-xG,ACmp,ACmp%,APrgDist,AXa,AKP,APPA,ACrsPA,ATB,ACrs,ACK,ASCA,AGCA,ATkIW,ALost,ABlocks,APass,AInt,AClr,AAttPen,ASucc%,ACPA,ADis,APrgR,AOnG,AOnGA,AOnxG,AOnxGA
0,0,5/8/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,3,1085,1100,1110,1110,1060,1090,76,77,75,76,4,2,26.6,25.2,1.00,0.76,0.97,1.02,0.96,0.75,45.8,14.2,32.2,71.8,24.3,3.51,-0.02,348.8,77.2,2310.4,0.61,8.19,6.14,1.27,0.95,15.4,4.89,19.4,1.76,11.20,6.73,12.3,9.08,9.65,21.5,19.9,46.9,5.41,12.10,32.1,1.05,1.30,1.02,1.29,4,1245,1285,1250,1250,1240,1320,80,83,81,79,7,7,24.1,23.0,2.14,1.62,2.05,1.87,1.79,1.39,59.5,21.7,54.2,70.6,35.1,5.03,0.27,482.8,83.1,2658.9,1.21,11.70,12.01,1.65,2.24,17.8,5.81,27.6,3.81,8.89,6.51,9.49,7.22,6.35,15.8,34.0,46.3,7.38,10.10,53.5,2.24,1.16,1.87,1.12
1,1,6/8/2022,12:30,Fulham,Liverpool,2,2,D,1,0,H,A Madley,3,1095,1100,1070,1065,1120,1140,76,78,76,74,2,1,27.6,26.0,1.38,0.86,0.74,1.20,1.03,0.75,48.8,17.3,35.2,76.6,24.3,3.65,0.18,367.4,77.4,2348.8,0.61,8.08,7.35,2.51,0.51,20.4,4.81,19.8,2.38,9.16,7.30,11.9,8.73,8.89,19.8,19.9,45.6,3.32,9.92,34.9,1.46,1.38,1.20,1.65,4,1220,1275,1180,1250,1260,1300,85,84,82,87,9,9,26.9,25.7,1.81,1.49,1.76,1.85,1.77,1.45,60.6,18.6,51.8,72.1,37.8,5.27,-0.04,526.5,82.2,3164.3,1.36,11.80,11.60,2.19,2.70,20.9,6.11,26.9,3.32,9.59,9.46,9.30,7.30,8.65,15.3,32.4,46.2,5.97,10.80,51.3,1.92,1.16,1.85,1.33
2,2,6/8/2022,15:00,Bournemouth,Aston Villa,2,0,H,1,0,H,P Bankes,2,1035,1095,1020,1110,1050,1080,74,75,73,73,2,1,25.8,25.5,1.00,0.65,1.00,1.03,1.03,0.76,39.8,13.2,26.1,67.7,21.6,3.38,-0.03,303.8,75.9,2159.2,0.70,6.92,6.27,1.97,1.30,13.4,3.81,16.3,1.73,9.84,9.24,13.3,8.73,9.38,27.0,18.5,44.1,3.84,9.78,25.6,1.00,1.89,1.03,1.70,3,1070,1100,1070,1075,1070,1130,79,79,78,80,5,3,25.8,25.2,1.27,0.89,1.19,1.28,1.20,0.98,49.5,16.7,32.8,72.8,32.4,3.78,-0.01,363.3,79.3,2320.0,0.81,8.38,7.08,2.11,1.38,14.8,4.30,19.6,2.16,8.97,8.78,11.50,8.32,8.43,19.1,19.6,42.0,4.57,9.78,32.5,1.32,1.22,1.28,1.38
3,3,6/8/2022,15:00,Leeds,Wolves,2,1,H,1,1,D,R Jones,3,1100,1090,1070,1075,1130,1120,76,78,76,76,3,2,24.2,24.3,1.19,0.81,1.16,1.24,1.18,0.83,46.8,14.4,37.1,61.2,13.5,3.76,-0.05,325.8,73.5,2205.0,0.71,8.78,6.86,1.59,1.11,16.5,5.05,20.3,1.81,12.60,11.80,14.1,10.60,10.40,19.6,20.4,39.0,3.92,10.90,36.8,1.27,2.00,1.24,1.76,3,1080,1090,1100,1100,1060,1080,79,79,81,77,5,4,25.9,23.6,0.76,0.32,0.68,0.98,0.92,0.68,50.0,16.1,35.7,69.3,29.7,3.22,-0.23,382.4,79.4,2299.3,0.55,7.81,6.73,1.89,1.32,17.5,4.89,18.9,1.19,9.89,8.43,13.20,8.57,7.49,21.5,18.2,48.3,3.76,8.59,35.4,0.84,1.43,0.98,1.54
4,4,6/8/2022,15:00,Newcastle,Nott'm Forest,2,0,H,0,0,D,S Hooper,4,1210,1260,1250,1300,1165,1220,78,78,76,79,5,3,26.6,25.9,1.70,1.11,1.54,1.90,1.77,1.37,52.7,18.2,42.9,75.6,37.8,4.86,-0.19,368.3,78.0,2399.1,1.19,11.50,10.70,1.73,2.38,19.7,7.22,26.6,2.81,9.57,8.68,11.7,8.81,8.78,17.5,30.8,44.3,5.89,9.54,42.4,1.81,0.86,1.90,1.01,3,1050,1100,1065,1100,1030,1105,76,75,76,74,1,1,24.7,25.9,0.95,0.57,0.86,1.05,0.92,0.70,37.7,11.5,22.8,65.9,21.6,2.89,-0.10,252.4,70.2,1914.6,0.55,6.54,4.59,1.32,0.84,14.1,3.35,15.7,1.41,10.20,7.59,13.20,9.11,9.27,24.9,16.2,45.1,2.92,7.22,22.5,1.00,1.81,1.05,1.70


In [ ]:
#data_teams = data.drop(['HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA', 'B365>2.5', 'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA', 'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA'], axis=1)
#data_teams.head()

In [ ]:
#data_teams.to_csv('../data/data_teams.csv')

In [ ]:
#data_betting = data.drop(['HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH', 'HSDA', 'HO', 'HA', 'HM', 'HD', 'HDP', 'HIP', 'HSAAg', 'HATAAg', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG', 'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA', 'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA', 'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA', 'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP', 'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK', 'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr', 'AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG', 'AOnGA', 'AOnxG', 'AOnxGA'], axis=1)
#data_betting.head()

In [ ]:
#data_betting.to_csv('../data/data_betting.csv')

In [40]:
X = data[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HDP', 'HIP', 'HSAAg', 'HATAAg', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG',
          'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA',
          'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA',
          'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA',
          'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP',
          'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK',
          'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr','AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG',
          'AOnGA', 'AOnxG', 'AOnxGA']].copy()
y = data['FTHG']

In [41]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [43]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

In [44]:
pipe.fit(X_train, y_train)

Iteration 1, loss = 2.53349387
Iteration 2, loss = 1.59185845
Iteration 3, loss = 1.11309534
Iteration 4, loss = 0.98417573
Iteration 5, loss = 1.05273778
Iteration 6, loss = 1.10770028
Iteration 7, loss = 1.07501950
Iteration 8, loss = 0.98767830
Iteration 9, loss = 0.90021532
Iteration 10, loss = 0.84561046
Iteration 11, loss = 0.82431371
Iteration 12, loss = 0.81886998
Iteration 13, loss = 0.81927883
Iteration 14, loss = 0.81322616
Iteration 15, loss = 0.80066116
Iteration 16, loss = 0.78436798
Iteration 17, loss = 0.76549221
Iteration 18, loss = 0.75661672
Iteration 19, loss = 0.75372342
Iteration 20, loss = 0.74882599
Iteration 21, loss = 0.74066215
Iteration 22, loss = 0.73083823
Iteration 23, loss = 0.72093089
Iteration 24, loss = 0.71407051
Iteration 25, loss = 0.70902601
Iteration 26, loss = 0.70413923
Iteration 27, loss = 0.69838999
Iteration 28, loss = 0.69166104
Iteration 29, loss = 0.68226252
Iteration 30, loss = 0.67684322
Iteration 31, loss = 0.67098072
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [45]:
print(f'R2 Score: {r2_score(y_test, pipe.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2 Score: -0.1820293829643509
MAE: 1.228311016502883


In [46]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 1.36753882
Iteration 2, loss = 0.99038673
Iteration 3, loss = 0.98203171
Iteration 4, loss = 0.90118719
Iteration 5, loss = 0.82427793
Iteration 6, loss = 0.79856237
Iteration 7, loss = 0.78508483
Iteration 8, loss = 0.75976367
Iteration 9, loss = 0.75723239
Iteration 10, loss = 0.75117604
Iteration 11, loss = 0.72226032
Iteration 12, loss = 0.70963579
Iteration 13, loss = 0.69524419
Iteration 14, loss = 0.67532700
Iteration 15, loss = 0.67323584
Iteration 16, loss = 0.66136452
Iteration 17, loss = 0.63772699
Iteration 18, loss = 0.62176923
Iteration 19, loss = 0.60974524
Iteration 20, loss = 0.59510861
Iteration 21, loss = 0.57855776
Iteration 22, loss = 0.56484377
Iteration 23, loss = 0.55088232
Iteration 24, loss = 0.53604141
Iteration 25, loss = 0.52090474
Iteration 26, loss = 0.51131110
Iteration 27, loss = 0.49853601
Iteration 28, loss = 0.47996181
Iteration 29, loss = 0.46275573
Iteration 30, loss = 0.45326626
Iteration 31, loss = 0.44306589
Iteration 32, los

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [47]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.21447498939065812
MAE: 1.2563012229752528


In [48]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (120, 120, 120),
                            alpha = 24))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 34.14909815
Iteration 2, loss = 32.59917289
Iteration 3, loss = 31.29143004
Iteration 4, loss = 30.19637220
Iteration 5, loss = 29.19820828
Iteration 6, loss = 28.19398250
Iteration 7, loss = 27.17553405
Iteration 8, loss = 26.20227669
Iteration 9, loss = 25.28279819
Iteration 10, loss = 24.39009690
Iteration 11, loss = 23.52098746
Iteration 12, loss = 22.66804997
Iteration 13, loss = 21.83990608
Iteration 14, loss = 21.03389650
Iteration 15, loss = 20.26631834
Iteration 16, loss = 19.52217611
Iteration 17, loss = 18.80180084
Iteration 18, loss = 18.10358736
Iteration 19, loss = 17.43022448
Iteration 20, loss = 16.78017947
Iteration 21, loss = 16.15266236
Iteration 22, loss = 15.54785603
Iteration 23, loss = 14.96290805
Iteration 24, loss = 14.39978645
Iteration 25, loss = 13.85672862
Iteration 26, loss = 13.33371334
Iteration 27, loss = 12.82969311
Iteration 28, loss = 12.34421924
Iteration 29, loss = 11.87614055
Iteration 30, loss = 11.42663924
Iteration 31, loss 

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=24, hidden_layer_sizes=(120, 120, 120),
                              verbose=True))])

In [49]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: 0.1561870122833673
MAE: 1.0663141810905852


In [50]:
X = data[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HDP', 'HIP', 'HSAAg', 'HATAAg', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG',
          'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA',
          'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA',
          'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA',
          'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP',
          'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK',
          'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr','AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG',
          'AOnGA', 'AOnxG', 'AOnxGA']].copy()
y = data['FTAG']

In [51]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [53]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 1.15145599
Iteration 2, loss = 0.74188185
Iteration 3, loss = 0.69049931
Iteration 4, loss = 0.73514695
Iteration 5, loss = 0.71079887
Iteration 6, loss = 0.63887520
Iteration 7, loss = 0.58828060
Iteration 8, loss = 0.57488055
Iteration 9, loss = 0.58136387
Iteration 10, loss = 0.58301611
Iteration 11, loss = 0.56873996
Iteration 12, loss = 0.54848823
Iteration 13, loss = 0.53633321
Iteration 14, loss = 0.53531925
Iteration 15, loss = 0.53137335
Iteration 16, loss = 0.52118913
Iteration 17, loss = 0.50787288
Iteration 18, loss = 0.50060483
Iteration 19, loss = 0.49571474
Iteration 20, loss = 0.48947777
Iteration 21, loss = 0.48275283
Iteration 22, loss = 0.47585954
Iteration 23, loss = 0.47051162
Iteration 24, loss = 0.46645964
Iteration 25, loss = 0.46049585
Iteration 26, loss = 0.45646149
Iteration 27, loss = 0.45087792
Iteration 28, loss = 0.44564825
Iteration 29, loss = 0.44258054
Iteration 30, loss = 0.43935799
Iteration 31, loss = 0.43644150
Iteration 32, los

C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr', MLPRegressor(verbose=True))])

In [54]:
print(f'R2 Score: {r2_score(y_test, pipe.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2 Score: -0.2770124005357213
MAE: 1.0730602259000064


In [55]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (100, 100, 100),
                            max_iter = 1000))
    ]
)

pipe.fit(X_train, y_train)

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Iteration 1, loss = 1.19113175
Iteration 2, loss = 0.77418474
Iteration 3, loss = 0.68291093
Iteration 4, loss = 0.71923153
Iteration 5, loss = 0.64284715
Iteration 6, loss = 0.59381135
Iteration 7, loss = 0.59530091
Iteration 8, loss = 0.59909229
Iteration 9, loss = 0.57862956
Iteration 10, loss = 0.55418818
Iteration 11, loss = 0.54302739
Iteration 12, loss = 0.54165674
Iteration 13, loss = 0.53081660
Iteration 14, loss = 0.51811003
Iteration 15, loss = 0.51215449
Iteration 16, loss = 0.50410621
Iteration 17, loss = 0.49279425
Iteration 18, loss = 0.48668141
Iteration 19, loss = 0.47781131
Iteration 20, loss = 0.46803042
Iteration 21, loss = 0.45979796
Iteration 22, loss = 0.45086432
Iteration 23, loss = 0.44321456
Iteration 24, loss = 0.43334891
Iteration 25, loss = 0.42401602
Iteration 26, loss = 0.41501584
Iteration 27, loss = 0.40442250
Iteration 28, loss = 0.39755601
Iteration 29, loss = 0.38844838
Iteration 30, loss = 0.37922636
Iteration 31, loss = 0.36827878
Iteration 32, los

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000,
                              verbose=True))])

In [56]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: -0.4242443704294785
MAE: 1.1244354226939748


In [57]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )),
        ('scaler', MinMaxScaler()),
        ('mlr', MLPRegressor(verbose = True,
                            hidden_layer_sizes = (24, 24, 24),
                            alpha = 30))
    ]
)

pipe.fit(X_train, y_train)

Iteration 1, loss = 11.01345436
Iteration 2, loss = 10.71237083
Iteration 3, loss = 10.43516305
Iteration 4, loss = 10.17643461
Iteration 5, loss = 9.93117133
Iteration 6, loss = 9.69435557
Iteration 7, loss = 9.47250509
Iteration 8, loss = 9.26213843
Iteration 9, loss = 9.06119966
Iteration 10, loss = 8.87140595
Iteration 11, loss = 8.68762036
Iteration 12, loss = 8.50648193
Iteration 13, loss = 8.32966896
Iteration 14, loss = 8.15524230
Iteration 15, loss = 7.98412518
Iteration 16, loss = 7.81648010
Iteration 17, loss = 7.65410789
Iteration 18, loss = 7.49341093
Iteration 19, loss = 7.33838339
Iteration 20, loss = 7.18689556
Iteration 21, loss = 7.03957151
Iteration 22, loss = 6.89525997
Iteration 23, loss = 6.75461133
Iteration 24, loss = 6.61707254
Iteration 25, loss = 6.48253701
Iteration 26, loss = 6.35194755
Iteration 27, loss = 6.22344871
Iteration 28, loss = 6.09857652
Iteration 29, loss = 5.97651541
Iteration 30, loss = 5.85753979
Iteration 31, loss = 5.74134295
Iteration 32,

C:\Users\17202\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\17202\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(sparse=False),
                                                  ['TeamH', 'TeamA',
                                                   'Referee'])])),
                ('scaler', MinMaxScaler()),
                ('mlr',
                 MLPRegressor(alpha=30, hidden_layer_sizes=(24, 24, 24),
                              verbose=True))])

In [58]:
print(f'R2: {r2_score(y_test, pipe.predict(X_test))}')

print(f'MAE: {mean_absolute_error(y_test, pipe.predict(X_test))}')

R2: 0.009588341479838225
MAE: 0.9306236428952744
